In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
import os
os.chdir(r'C:\Users\newbi\project-local\WM0824TU-economics-of-cybersecurity-DDoS')

In [4]:
# data source: https://surfdrive.surf.nl/files/index.php/s/CZWfWQp3VKGKa8m
df = pd.read_csv('SecurityMetrics/data_with_ports.csv.gz', sep='|', compression='gzip')

C:\Users\newbi\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2785: DtypeWarning: Columns (15,17,19,27) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [5]:
# parse string to datetime format
df['date'] = pd.to_datetime(df['date'], format='%Y-%m-%d')
df['start_time'] = pd.to_datetime(df['start_time'], format='%Y-%m-%dT%H:%M:%S+09:00')
df['stop_time'] = pd.to_datetime(df['stop_time'], format='%Y-%m-%dT%H:%M:%S+09:00')

# Data Preparation

In [12]:
# remove reserved ips

reserved_ips = [
    "0.",       # software
    "10.",      # private network
    "100.",     # private network
    "169.254.", # subnet
    "172.",     # private network
    "192."      # private network
]

for ips in reserved_ips:
    df = df[~(df.org.isna() & df.target_ip.str.startswith(ips))]

In [19]:
df["duration"]

0          297.0
1          298.0
2           29.0
3          358.0
4          188.0
5           12.0
6           15.0
7          221.0
8           11.0
9            9.0
10         217.0
11          10.0
12          90.0
13         113.0
14           7.0
15           6.0
16           5.0
17          65.0
18         221.0
19          44.0
20          33.0
21          13.0
22          34.0
23         129.0
24         290.0
25         122.0
26         100.0
27         621.0
28         769.0
29         301.0
           ...  
5721402    296.0
5721403    296.0
5721404    754.0
5721405    205.0
5721406    205.0
5721407    205.0
5721408      3.0
5721409     22.0
5721410     16.0
5721411    527.0
5721412     22.0
5721413     17.0
5721414    527.0
5721415     22.0
5721416      8.0
5721417      8.0
5721418     31.0
5721419     31.0
5721420     18.0
5721421     15.0
5721422    451.0
5721423     13.0
5721424     19.0
5721425     25.0
5721426     15.0
5721427      5.0
5721428     60.0
5721429     22

In [27]:
# size of victims
# victim org
# country victim
# type of victim

df_clean = df[["target_ip", "date", "service", "duration", "packets", "raw_country", "as_type", 
               "org", "org_ipsize_seen", "org_tag", "year"]]
df_clean[~df_clean["org"].isna()]

In [49]:
occurence_attacks = pd.DataFrame(df_clean["org"].value_counts().rename("count_attacks"))
df_clean = df_clean.merge(occurence_attacks, left_on="org", right_index=True)

#Remove companies that received just 1 attack, as we assume the first attack was only a trial
df_clean = df_clean[df_clean["count_attacks"] > 1]

In [57]:
df_clean

,target_ip,date,service,duration,packets,raw_country,as_type,org,org_ipsize_seen,org_tag,year,count_attacks
0,1.0.131.184,2014-08-02,dns,297.0,609,Thailand,isp-broadband,TOT,556359.0,NaN,2014,2129
1,1.0.131.184,2014-08-02,dns,298.0,602,Thailand,isp-broadband,TOT,556359.0,NaN,2014,2129
461,1.10.255.20,2014-11-14,dns,225.0,115,Thailand,isp-broadband,TOT,556359.0,NaN,2014,2129
462,1.10.255.20,2014-11-14,dns,224.0,115,Thailand,isp-broadband,TOT,556359.0,NaN,2014,2129
463,1.10.255.20,2014-11-14,dns,224.0,115,Thailand,isp-broadband,TOT,556359.0,NaN,2014,2129
1493,1.2.208.181,2014-01-29,chg,621.0,21467,Thailand,isp-broadband,TOT,556359.0,NaN,2014,2129
1494,1.2.211.7,2014-01-31,chg,1802.0,25997,Thailand,isp-broadband,TOT,556359.0,NaN,2014,2129
1495,1.2.211.7,2014-02-04,chg,1803.0,162259,Thailand,isp-broadband,TOT,556359.0,NaN,2014,2129
1496,1.2.211.78,2014-01-29,chg,1802.0,62546,Thailand,isp-broadband,TOT,556359.0,NaN,2014,2129
1497,1.2.212.156,2014-01-30,chg,1802.0,61779,Thailand,isp-broadband,TOT,556359.0,NaN,2014,2129


In [59]:
df_clean.groupby("year").count()

,target_ip,date,service,duration,packets,raw_country,as_type,org,org_ipsize_seen,org_tag,count_attacks
year,,,,,,,,,,,
2014,1048978,1048978,1048978,1048978,1048978,1048978,767848,1048978,1010664,598632,1048978
2015,4658810,4658810,4658810,4658810,4658810,4658810,2888088,4658810,4651451,2623409,4658810


In [25]:
df_host = df[df["org_is_hosting"] == 1]
df_host["org_tag"].unique()

array([nan, 'isp-mobile', 'isp-broadband', 'isp-other', 'hosting', 'cdn'],
      dtype=object)

In [18]:
df["org_tag"].unique()

array([nan, 'other', 'isp-mobile', 'isp-broadband', 'isp-other',
       'hosting', 'cdn', 'edu', 'gov'], dtype=object)

In [14]:
df.columns

Index(['target_ip', 'date', 'sensor_id', 'service', 'start_time', 'stop_time',
       'duration', 'packets', 'raw_country', 'raw_as', 'raw_hostname',
       'udp_port_list', 'pyasn_as', 'pyasn_as_bgp_size', 'cc', 'region',
       'is_oecd', 'as_type', 'as_type_confidence', 'tg_op', 'org', 'org_range',
       'org_rangesize', 'org_ipsize_seen', 'org_domainsize_seen',
       'org_ipsize_seen_shared', 'org_is_hosting', 'org_tag', 'caida_type',
       'dc', 'subs', 'as_ipsize_seen', 'as_domainsize_seen', 'pps', 'year'],
      dtype='object')

# Normalise data based on size

In [10]:
df_host = df[df["as_type"] == "hosting"]
count_attacks = pd.DataFrame(df_host["org"].value_counts())

In [9]:
df[df["target_ip"] == '183.60.110.194']

,target_ip,date,sensor_id,service,start_time,stop_time,duration,packets,raw_country,raw_as,...,org_ipsize_seen_shared,org_is_hosting,org_tag,caida_type,dc,subs,as_ipsize_seen,as_domainsize_seen,pps,year
2736362,183.60.110.194,2015-09-29,sensor001,chg,2015-09-29 15:16:33,2015-09-29 15:17:12,39.0,186,China,AS58543 Guangdong,...,3130.0,1.0,NaN,Content,1,NaN,50860.0,287359.0,4.769231,2015
2736363,183.60.110.194,2015-09-29,sensor001,ntp,2015-09-29 15:16:33,2015-09-29 15:17:25,52.0,3166,China,AS58543 Guangdong,...,3130.0,1.0,NaN,Content,1,NaN,50860.0,287359.0,60.884615,2015
2736364,183.60.110.194,2015-09-29,sensor003,chg,2015-09-29 15:16:33,2015-09-29 15:17:20,47.0,236,China,AS58543 Guangdong,...,3130.0,1.0,NaN,Content,1,NaN,50860.0,287359.0,5.021277,2015
2736365,183.60.110.194,2015-09-29,sensor003,ntp,2015-09-29 15:16:34,2015-09-29 15:17:25,51.0,1600,China,AS58543 Guangdong,...,3130.0,1.0,NaN,Content,1,NaN,50860.0,287359.0,31.372549,2015
2736366,183.60.110.194,2015-09-29,sensor004,chg,2015-09-29 15:16:34,2015-09-29 15:17:20,46.0,185,China,AS58543 Guangdong,...,3130.0,1.0,NaN,Content,1,NaN,50860.0,287359.0,4.021739,2015
2736367,183.60.110.194,2015-09-29,sensor004,ntp,2015-09-29 15:16:33,2015-09-29 15:17:24,51.0,2158,China,AS58543 Guangdong,...,3130.0,1.0,NaN,Content,1,NaN,50860.0,287359.0,42.313725,2015
2736368,183.60.110.194,2015-09-29,sensor005,ntp,2015-09-29 15:16:33,2015-09-29 15:17:21,48.0,1090,China,AS58543 Guangdong,...,3130.0,1.0,NaN,Content,1,NaN,50860.0,287359.0,22.708333,2015
2736369,183.60.110.194,2015-09-30,sensor001,chg,2015-09-30 23:27:03,2015-09-30 23:29:05,122.0,100,China,AS58543 Guangdong,...,3130.0,1.0,NaN,Content,1,NaN,50860.0,287359.0,0.819672,2015
2736370,183.60.110.194,2015-09-30,sensor001,chg,2015-09-30 23:40:12,2015-09-30 23:42:17,125.0,394,China,AS58543 Guangdong,...,3130.0,1.0,NaN,Content,1,NaN,50860.0,287359.0,3.152000,2015
2736371,183.60.110.194,2015-09-30,sensor001,ntp,2015-09-30 14:45:08,2015-09-30 14:49:40,272.0,5603,China,AS58543 Guangdong,...,3130.0,1.0,NaN,Content,1,NaN,50860.0,287359.0,20.599265,2015
